# Capacitated Facility Location Problem

In [2]:
from pulp import *

In [3]:
# Lists (sets / Array) of Customers and Facilities
Customer = [1,2,3,4,5]
Facility = ['Fac-1', 'Fac-2', 'Fac-3']

In [4]:
# Dictionaries of the demands and facilities as well as fixed cost at each facility
Demand = {1 : 80, 2 : 270, 3 : 250, 4 : 160, 5 : 180}
Max_Supply = {'Fac-1' : 500, 'Fac-2' : 500, 'Fac-3' : 500} 
fixed_cost = {'Fac-1' : 1000, 'Fac-2' : 1000, 'Fac-3' : 1000 }

In [5]:
# Dictionary of the transportation cost from each facility to all the customers 
transportation_cost = {'Fac-1' : {1 : 4, 2 : 5, 3 : 6, 4 : 8, 5 : 10},
                       'Fac-2' : {1 : 6, 2 : 4, 3 : 3, 4 : 5, 5 : 8},
                       'Fac-3' : {1 : 9, 2 : 7, 3 : 4, 4 : 3, 5 : 4}
                      }

In [6]:
# Setting the Problem
prob = LpProblem("Capacitated_Facility_Location_Problem", LpMinimize)

/Users/wenyuc/opt/anaconda3/envs/bioze_env/lib/python3.10/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [7]:
# Defining our Desicion Variables
use_facility = LpVariable.dicts("Use Facility", Facility, 0, 1, LpBinary)
ser_customer = LpVariable.dicts("Service", [(i,j) for i in Customer for j in Facility], 0)

In [8]:
# Setting the Objective Function
prob += lpSum(fixed_cost[j]*use_facility[j] for j in Facility) + lpSum(transportation_cost[j][i]*ser_customer[(i,j)] for j in Facility for i in Customer)

In [9]:
# Costraints
for i in Customer:
    prob += lpSum(ser_customer[(i,j)] for j in Facility) == Demand[i]

for j in Facility:
    prob += lpSum(ser_customer[(i,j)] for i in Customer) <= Max_Supply[j]*use_facility[j]

for i in Customer:
    for j in Facility:
        prob += ser_customer[(i,j)] <= Demand[i]*use_facility[j]

In [10]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/wenyuc/opt/anaconda3/envs/bioze_env/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/w_/w1t73xzx35j4jtrj94fjskj80000gn/T/4cc8a5c4e994486290aeb8d9625f9839-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/w_/w1t73xzx35j4jtrj94fjskj80000gn/T/4cc8a5c4e994486290aeb8d9625f9839-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 28 COLUMNS
At line 116 RHS
At line 140 BOUNDS
At line 144 ENDATA
Problem MODEL has 23 rows, 18 columns and 63 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 5610 - 0.00 seconds
Cgl0004I processed model has 23 rows, 18 columns (3 integer (3 of which binary)) and 63 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 2.22045e-16
Cbc0038I Solution found of 5610
Cbc0038I Relaxing continuous gives 5610
Cbc0038I Bef

1

In [16]:
# Initialize lists to store assignment information
assigned_farms = {j: [] for j in Facility}

# Collect assigned farms
for i in Facility:
    for j in Customer:
        if ser_customer[i, j].varValue == 1:
            assigned_farms[j].append(i)


KeyError: ('Fac-1', 1)

In [11]:
print("Solution Status = ", LpStatus[prob.status])

Solution Status =  Optimal


In [12]:
# Print the solution of Binary Decision Variables
Tolerance = 0.0001
for j in Facility:
    if use_facility[j].varValue > Tolerance:
        print("Estalish Facility at site = ", j)

Estalish Facility at site =  Fac-2
Estalish Facility at site =  Fac-3


In [13]:
# Print the solution of Continuous Decision Variables
for v in prob.variables():
    print(v.name, "=", v.varValue)

Service_(1,_'Fac_1') = 0.0
Service_(1,_'Fac_2') = 80.0
Service_(1,_'Fac_3') = 0.0
Service_(2,_'Fac_1') = 0.0
Service_(2,_'Fac_2') = 270.0
Service_(2,_'Fac_3') = 0.0
Service_(3,_'Fac_1') = 0.0
Service_(3,_'Fac_2') = 150.0
Service_(3,_'Fac_3') = 100.0
Service_(4,_'Fac_1') = 0.0
Service_(4,_'Fac_2') = 0.0
Service_(4,_'Fac_3') = 160.0
Service_(5,_'Fac_1') = 0.0
Service_(5,_'Fac_2') = 0.0
Service_(5,_'Fac_3') = 180.0
Use_Facility_Fac_1 = 0.0
Use_Facility_Fac_2 = 1.0
Use_Facility_Fac_3 = 1.0


In [14]:
# Print Optimal
print("Total Cost = ", value(prob.objective))

Total Cost =  5610.0
